<img src="https://media.theresanaiforthat.com/icons/mistral-ai.svg?height=207" width="30" height="30">
<h1 align = "center">Automated Information Watch via AI</h1>

---

This notebook is an essay on the use of Mistral.AI for an information watch on the internet.
The goal is to retrieve articles from the internet and make a summary using `mistral.ai`, then send them by email in HTML format generated by `Mistral.ai`

The code is divided into 3 parts:

- retrieving articles via url with `BeautifulSoup`
- generating summaries with `Mistral.AI`
- generating the HTML content to send it by automated email via `Mistral.AI`
- sending the email message

We will detail the code step by step in the rest of the notebook.

## Loading Packages

In [146]:
import requests
from bs4 import BeautifulSoup
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## Article Extraction
Once the packages are loaded, we will write the code to retrieve the different articles that we want to summarize.<br><br>
Each article is then extracted based on the URL address available in the BeautifulSoup object and stored in a dataframe named `Messages` which will allow the AI to process the articles individually:

In [239]:
response = requests.get("https://www.oppbtp.com")
soup = BeautifulSoup(response.text, 'html.parser')
articles = soup.find_all('a', class_ = "card card-column card-light no-decoration")

Messages = []

for article in articles:
    text = article.get("href")
    test = requests.get(text)
    Messages.append(BeautifulSoup(test.text, 'html.parser').find('div', {'class':"entry-content"}).text.replace('\n', ' ').strip())

## Summary Generation
We define the role of our AI to ask it to provide us with a summary of the article that we will give it as input:

In [240]:
messages = [
    ChatMessage(role="system", content="Tu es un programme de génération de résumé à partir de texte que l'on te donne, tu ne fais que fournir un résumé du texte que l'on te donne en input")
]

Once the role is defined, for each Message, we will ask the program to provide us with the summary of the article:

In [241]:
model = "mistral-medium-latest"
api_key="dXhF7z625frqL76yHgWY5PZJ9l0298mg" # ne fonctionne plus après le 31 mars 2024

responses = []

for message in Messages:
    messages.append(ChatMessage(role="user", content=message))
    client = MistralClient(api_key=api_key)
    Mistral = client.chat(model=model, messages=messages)
    responses.append(Mistral.choices[0].message.content.replace('\n', ' '))
    messages.pop(-1)

All responses will have to be grouped into a single string in order to ask the AI to generate an email based on all the summaries it previously generated:

In [242]:
test = ""

for i in range(len(responses)):
    test = test + "- titre de l'article : " + articles[i].find("h4").text+", résumé de l'article : "+ responses[i] + "\n"

## Email Body Generation
A new role is defined for the model here, the goal is to automatically generate an email body in HTML format to generate the formatting and send it later to the chosen recipient:

In [274]:
Mailer = [
    ChatMessage(role="system", content="tu es un assistant de creation de mail, tu ne reponds que par un corps de mail mis en page, étant un mail contenant les informations dans le message que tu reçois, tu commenceras par un message de politesse de 20 mots pour dire bonjour, de façon amicale, je veux simplement le corps du texte, sans titre. tu réponds au format HTML.")
]

Once the role is defined, we will ask the program to provide us with the email body:

In [275]:
model = "mistral-medium-latest"
api_key="dXhF7z625frqL76yHgWY5PZJ9l0298mg" # ne fonctionne plus après le 31 mars 2024

Mailer.append(ChatMessage(role="user", content=test))
client = MistralClient(api_key=api_key)
Mistral = client.chat(model=model, messages=Mailer)
responses = Mistral.choices[0].message.content

The email sending function is defined:

In [280]:
def send_mail(From : str, to: str, password_google : str):
    msg = MIMEMultipart()
    msg['From'] = From
    msg['To'] = to
    msg['Subject'] = 'résumé d\'article Mistral.AI'
    body = responses

    msg.attach(MIMEText(body, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(From, password_google)
    server.sendmail(From, to, msg.as_string())
    server.quit()

The email sending function is executed:

In [281]:
send_mail('*****@gmail.com','*****@gmail.com', '*****')

**Conclusion** : This program is an example of the use of Mistral.AI, it is possible to use it for other projects. It is possible to retrieve text from another source, and thus work on a different type of information. It will be necessary to think about how to extract the data, which may be different from one site to another.